In [236]:
# Imports
import pandas as pd
from pathlib import Path
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import hvplot.pandas
from fractions import Fraction
pd.set_option('display.max_columns', None)

In [237]:
# Import dataframes
results_df = pd.read_csv(Path('../Exports/week_6_results.csv'))

In [238]:
results_df

,week,away_name,home_name,away_team_win_%,home_team_win_%,predicted_winner,actual_winner
0,6,Arizona Cardinals,Cleveland Browns,0.685903,0.314097,Arizona Cardinals,Arizona Cardinals
1,6,Buffalo Bills,Tennessee Titans,0.679792,0.320208,Buffalo Bills,Tennessee Titans
2,6,Cincinnati Bengals,Detroit Lions,0.820665,0.179335,Cincinnati Bengals,Cincinnati Bengals
3,6,Dallas Cowboys,New England Patriots,0.780525,0.219475,Dallas Cowboys,Dallas Cowboys
4,6,Green Bay Packers,Chicago Bears,0.968273,0.031727,Green Bay Packers,Green Bay Packers
5,6,Houston Texans,Indianapolis Colts,0.087965,0.912035,Indianapolis Colts,Indianapolis Colts
6,6,Kansas City Chiefs,Washington Football Team,0.983081,0.016919,Kansas City Chiefs,Kansas City Chiefs
7,6,Las Vegas Raiders,Denver Broncos,0.202403,0.797597,Denver Broncos,Las Vegas Raiders
8,6,Los Angeles Chargers,Baltimore Ravens,0.092000,0.908000,Baltimore Ravens,Baltimore Ravens
9,6,Los Angeles Rams,New York Giants,0.816127,0.183873,Los Angeles Rams,Los Angeles Rams


In [239]:
#American Odds Converter
def american_odds_converter(away_team_win_pct, home_team_win_pct):
    if home_team_win_pct - away_team_win_pct > 0:
        home_team_american_odds = round(((home_team_win_pct/(1-home_team_win_pct))*100)*(-1),0)
        away_team_american_odds = round(((100/(away_team_win_pct))-100),0)
    elif home_team_win_pct - away_team_win_pct < 0:
        away_team_american_odds = round(((away_team_win_pct/(1-away_team_win_pct))*100)*(-1),0)
        home_team_american_odds = round(((100/(home_team_win_pct))-100),0)
    else:
        away_team_american_odds = 100
        home_team_american_odds = 100
    return away_team_american_odds, home_team_american_odds

#Fractional Odds Converter
def fractional_odds_converter(away_team_win_pct, home_team_win_pct):
    away_team_american_odds, home_team_american_odds = american_odds_converter(away_team_win_pct, home_team_win_pct)
    if home_team_american_odds < 0:
        away_team_fractional_odds = (Fraction(round(away_team_american_odds / 100, 0)))
        home_team_fractional_odds = (Fraction(round(((-100)/home_team_american_odds)*8)/8))
    elif home_team_american_odds > 0:
        home_team_fractional_odds = (Fraction(round(home_team_american_odds / 100, 0)))
        away_team_fractional_odds = (Fraction(round(((-100)/away_team_american_odds)*8)/8))
    else: 
        away_team_fractional_odds = Fraction(1)
        home_team_fractional_odds = Fraction(1)
    return away_team_fractional_odds, home_team_fractional_odds     

#Decimal Odds Converter
def decimal_odds_converter(away_team_win_pct, home_team_win_pct):
    away_team_american_odds, home_team_american_odds = american_odds_converter(away_team_win_pct, home_team_win_pct)
    if home_team_american_odds < 0:
        away_team_decimal_odds = round((1 + (away_team_american_odds/100)),2)
        home_team_decimal_odds = round((1-(100/home_team_american_odds)),2)
    elif home_team_american_odds > 0:
        home_team_decimal_odds = round((1 + (home_team_american_odds/100)),2)
        away_team_decimal_odds = round((1-(100/away_team_american_odds)),2)
    else: 
        away_team_decimal_odds = 1
        home_team_decimal_odds = 1
    return away_team_decimal_odds, home_team_decimal_odds      

In [240]:
away_team_american_odds, home_team_american_odds = american_odds_converter(results_df.iloc[7]['away_team_win_%'], results_df.iloc[7]['home_team_win_%'])

away_team_fractional_odds, home_team_fractional_odds = fractional_odds_converter(results_df.iloc[7]['away_team_win_%'], results_df.iloc[7]['home_team_win_%'])

away_team_decimal_odds, home_team_decimal_odds = decimal_odds_converter(results_df.iloc[7]['away_team_win_%'], results_df.iloc[7]['home_team_win_%'])

display(results_df.iloc[7]['away_team_win_%'])
display(results_df.iloc[7]['home_team_win_%'])
display(away_team_american_odds)
display(home_team_american_odds)
display(away_team_fractional_odds)
display(home_team_fractional_odds)
display(away_team_decimal_odds)
display(home_team_decimal_odds)


0.2024031600939235

0.7975968399060764

394.0

-394.0

Fraction(4, 1)

Fraction(1, 4)

4.94

1.25

In [241]:
projected_odds = results_df.copy()

In [242]:
probability = results_df.drop(columns = ['week','away_name' ,'home_name', 'predicted_winner', 'actual_winner'])

In [243]:
probability['American Odds (Away Team, Home Team)'] = probability.apply(lambda x: american_odds_converter(x['away_team_win_%'], x['home_team_win_%']), axis=1)

In [244]:
probability['Fractional Odds (Away Team, Home Team)'] = probability.apply(lambda x: fractional_odds_converter(x['away_team_win_%'], x['home_team_win_%']), axis=1)

In [245]:
probability['Decimal Odds (Away Team, Home Team)'] = probability.apply(lambda x: decimal_odds_converter(x['away_team_win_%'], x['home_team_win_%']), axis=1)

In [246]:
probability

,away_team_win_%,home_team_win_%,"American Odds (Away Team, Home Team)","Fractional Odds (Away Team, Home Team)","Decimal Odds (Away Team, Home Team)"
0,0.685903,0.314097,"(-218.0, 218.0)","(1/2, 2)","(1.46, 3.18)"
1,0.679792,0.320208,"(-212.0, 212.0)","(1/2, 2)","(1.47, 3.12)"
2,0.820665,0.179335,"(-458.0, 458.0)","(1/4, 5)","(1.22, 5.58)"
3,0.780525,0.219475,"(-356.0, 356.0)","(1/4, 4)","(1.28, 4.56)"
4,0.968273,0.031727,"(-3052.0, 3052.0)","(0, 31)","(1.03, 31.52)"
5,0.087965,0.912035,"(1037.0, -1037.0)","(10, 1/8)","(11.37, 1.1)"
6,0.983081,0.016919,"(-5810.0, 5810.0)","(0, 58)","(1.02, 59.1)"
7,0.202403,0.797597,"(394.0, -394.0)","(4, 1/4)","(4.94, 1.25)"
8,0.092000,0.908000,"(987.0, -987.0)","(10, 1/8)","(10.87, 1.1)"
9,0.816127,0.183873,"(-444.0, 444.0)","(1/4, 4)","(1.23, 5.44)"


In [247]:
#results_df = pd.concat([results_df, probability.drop(columns=['away_team_win_%', 'home_team_win_%'])
results_df = pd.concat([results_df, probability.drop(columns = ['away_team_win_%','home_team_win_%'])], axis =1)

In [248]:
results_df

,week,away_name,home_name,away_team_win_%,home_team_win_%,predicted_winner,actual_winner,"American Odds (Away Team, Home Team)","Fractional Odds (Away Team, Home Team)","Decimal Odds (Away Team, Home Team)"
0,6,Arizona Cardinals,Cleveland Browns,0.685903,0.314097,Arizona Cardinals,Arizona Cardinals,"(-218.0, 218.0)","(1/2, 2)","(1.46, 3.18)"
1,6,Buffalo Bills,Tennessee Titans,0.679792,0.320208,Buffalo Bills,Tennessee Titans,"(-212.0, 212.0)","(1/2, 2)","(1.47, 3.12)"
2,6,Cincinnati Bengals,Detroit Lions,0.820665,0.179335,Cincinnati Bengals,Cincinnati Bengals,"(-458.0, 458.0)","(1/4, 5)","(1.22, 5.58)"
3,6,Dallas Cowboys,New England Patriots,0.780525,0.219475,Dallas Cowboys,Dallas Cowboys,"(-356.0, 356.0)","(1/4, 4)","(1.28, 4.56)"
4,6,Green Bay Packers,Chicago Bears,0.968273,0.031727,Green Bay Packers,Green Bay Packers,"(-3052.0, 3052.0)","(0, 31)","(1.03, 31.52)"
5,6,Houston Texans,Indianapolis Colts,0.087965,0.912035,Indianapolis Colts,Indianapolis Colts,"(1037.0, -1037.0)","(10, 1/8)","(11.37, 1.1)"
6,6,Kansas City Chiefs,Washington Football Team,0.983081,0.016919,Kansas City Chiefs,Kansas City Chiefs,"(-5810.0, 5810.0)","(0, 58)","(1.02, 59.1)"
7,6,Las Vegas Raiders,Denver Broncos,0.202403,0.797597,Denver Broncos,Las Vegas Raiders,"(394.0, -394.0)","(4, 1/4)","(4.94, 1.25)"
8,6,Los Angeles Chargers,Baltimore Ravens,0.092000,0.908000,Baltimore Ravens,Baltimore Ravens,"(987.0, -987.0)","(10, 1/8)","(10.87, 1.1)"
9,6,Los Angeles Rams,New York Giants,0.816127,0.183873,Los Angeles Rams,Los Angeles Rams,"(-444.0, 444.0)","(1/4, 4)","(1.23, 5.44)"
